# Initial familiarization with some files with data

In [7]:
import glob
import os

from pathlib import Path
import xml.etree.ElementTree as ET
# get list of files in data/raw


current_dir = os.path.dirname(os.path.realpath('.'))
project_dir = Path(current_dir).parents[0]
data_dir = os.path.join(project_dir, 'data')

list_of_files = glob.glob(os.path.join(data_dir, 'raw', '*.xml'))
len(list_of_files)

13300

In [19]:
def get_abstract(filename):
    xml_files = open(os.path.join(data_dir, 'raw', filename), 'r').read()

    root = ET.fromstring(xml_files)

 
    # Iterate through elements
    abstract = ''
    for child in root:
        # print(f'Tag: {child.tag}, Attributes: {child.attrib}')
        for subchild in child:
            if 'abstract' in str(subchild.tag).lower():
                abstract = subchild.text
                #print(f'  Sub-tag: {subchild.tag}, Text: {subchild.text[:]}')
    return abstract


In [23]:
dict_abstracts = {}
for f in list_of_files[:100]:
    try:
        abstract = get_abstract(f)
        paper = f.split('/')[-1].split('.')[0]

        dict_abstracts[paper] = abstract
    except Exception as e:
        print(f'Error with {f}')
        print(e)
        continue

dict_abstracts

{'2035358': "The NSF Convergence Accelerator supports team-based, multidisciplinary efforts that address challenges of national importance and show potential for deliverables in the near future. The objective of this workshop is to explore topics for potential NSF Convergence Accelerator tracks for FY 2021.&lt;br/&gt;&lt;br/&gt;The Future of Privacy Forum (FPF) will convene a workshop on “The Responsible Use of Data During Times of Crisis”, for pandemics as well as other crisis situations. The workshop will bring together US government leaders, international data protection authorities, corporate leaders, technologists, academic researchers and public health experts to examine benefits, risks, and strategies for the collection and protection of data in support of public health initiatives during crises, including for COVID-19. &lt;br/&gt;&lt;br/&gt;Ensuring that US and global stakeholders leading emergency efforts have the data-based knowledge, tools and governance structures to naviga